In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from typing import Optional, List
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser, JsonKeyOutputFunctionsParser
from langchain_groq import ChatGroq
from parser import *

In [3]:
model = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
)

In [4]:
pdf_path = "/home/duybaohuynhtan/Resume Analyzer/test.pdf"
content = read_pdf(pdf_path)

In [5]:
extraction_functions = [convert_pydantic_to_openai_function(ResumeSchema)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "ResumeSchema"})

/tmp/ipykernel_46275/2667766373.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  extraction_functions = [convert_pydantic_to_openai_function(ResumeSchema)]


In [6]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [7]:
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()

extraction_chain.invoke({"input": content})

{'education': [{'duration': '2024 - 2027',
   'institution': 'Borcelle University'},
  {'duration': '2021 - 2024', 'institution': 'Borcelle University'},
  {'duration': '2018 - 2021', 'institution': 'Borcelle University'}],
 'email': 'hello@reallygreatsite.com',
 'experiences': [{'company': 'Timmerman Industries',
   'duration': 'NOW-2024',
   'position': 'Office Manager'},
  {'company': 'Thynk Unlimited',
   'duration': '2023-2022',
   'position': 'Office Manager'},
  {'company': 'Ingoude Company',
   'duration': '2021-2020',
   'position': 'Office Manager'}],
 'name': 'RIAAN CHANDRAN',
 'phone': '+123-456-7890',
 'skills': ['Organizational Skills',
  'Leadership',
  'Time Management',
  'Communication Skills',
  'Problem Solving',
  'Hindi',
  'English']}